#### Install Libraries

In [47]:
! pip install livelossplot

### Import libraries

In [38]:
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow.keras as keras

import sys
sys.path.append('/home/jovyan/work')

import nima as nima


In [1]:
! pwd

/home/jovyan/work/notebooks


In [42]:
AVA_DATASET_DIR = '/home/jovyan/work/data/AVA/'
PROJECT_ROOT_DIR = '/home/jovyan/work/'
AVA_IMAGES_DIR = os.path.join(AVA_DATASET_DIR, 'images')
WEIGHTS_DIR = os.path.join(PROJECT_ROOT_DIR, )
assert os.path.isdir(AVA_DATASET_DIR), f'Invalid directory : {AVA_DATASET_DIR}'

from nima.utils.ava_dataset_utils import load_data, get_rating_columns

### Load Dataset

In [89]:
print(f'Project Root Directory : {PROJECT_ROOT_DIR}')

df_train, df_valid, df_test = load_data(AVA_DATASET_DIR, sample_size=1000)
print(f"Training length : {len(df_train)}, Validation length : {len(df_valid)}, Test size : {len(df_test)}")

Project Root Directory : /home/jovyan/work/
Number of samples picked 1000
Train length : 664, Validation length : 285, Test size : 50


In [31]:
df_train.head()

,image_id,count_rating_1,count_rating_2,count_rating_3,count_rating_4,count_rating_5,count_rating_6,count_rating_7,count_rating_8,count_rating_9,count_rating_10
0,123500,2,9,34,77,80,51,24,0,2,0
1,137078,1,3,4,32,78,43,17,3,1,2
2,136778,0,3,17,25,76,43,8,7,4,1
3,103641,2,4,8,23,37,65,53,43,29,18
4,103604,0,8,13,37,71,65,34,19,6,2


### Create Model

In [84]:
from nima.model.model_builder import NIMA

nima_cnn = NIMA(base_model_name='vgg', metrics=['accuracy'])

print(f'Model class name in notebook : {nima_cnn.base_model_name}')
nima_cnn.build()
nima_cnn.model.summary()

nima_cnn.compile()

Model module - tensorflow.keras.applications.vgg19.VGG19
Model class name in notebook : VGG19
Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_______________________________

### Create Datagenerators

In [45]:
from nima.model.data_generator import NimaDataGenerator

x_col, y_cols = 'image_id', get_rating_columns()

train_datagen = NimaDataGenerator(
    df_train,
    AVA_IMAGES_DIR,
    x_col,
    y_cols,
    nima_cnn.preprocess_input,
    is_train=True,
    batch_size=32,
)
valid_datagen = NimaDataGenerator(
    df_valid,
    AVA_IMAGES_DIR,
    x_col,
    y_cols,
    nima_cnn.preprocess_input,
    is_train=True,
    batch_size=32,
)
test_datagen = NimaDataGenerator(
    df_test,
    AVA_IMAGES_DIR,
    x_col,
    None,
    nima_cnn.preprocess_input,
    is_train=False,
    batch_size=32,
)

Found 664 valid image filenames belonging to 10 classes.
Found 285 valid image filenames belonging to 10 classes.
Found 50 valid image filenames belonging to 10 classes.


In [94]:
len(train_datagen)

664


21

In [95]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from livelossplot.inputs.keras import PlotLossesCallback
import time

arg_verbose, arg_epochs = 1, 32

# set model weight and path
weight_filename = f'{nima_cnn.base_model_name}_weight_best'
weight_filepath = os.path.join(WEIGHTS_DIR, weight_filename)
print(f'Model Weight path : {weight_filepath}')

es = EarlyStopping(monitor='val_loss', patience=4, verbose=arg_verbose)
ckpt = ModelCheckpoint(
    filepath=weight_filepath,
    save_weights_only=True,
    monitor="val_earth_movers_distance",
    mode="auto",
    save_best_only=True,
)
lr = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1)
plot_loss = PlotLossesCallback()

# start training
start_time = time.perf_counter()
history = nima_cnn.model.fit(train_datagen, validation_data=valid_datagen,
    epochs=arg_epochs, callbacks=[es, ckpt, lr, plot_loss],
    verbose=arg_verbose)
end_time = time.perf_counter()
print(f'Time taken : {time.strftime("%H:%M:%S", time.gmtime(end_time-start_time))}')

result_df = pd.DataFrame(history.history)

Model Weight path : /home/jovyan/work/VGG19_weight_best


TypeError: 'int' object is not iterable

In [ ]:
result_df.head()